In [1]:
import pandas as pd
import numpy as np

c:\Users\alber\anaconda3\envs\tensorflow-gpu\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\alber\anaconda3\envs\tensorflow-gpu\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\alber\anaconda3\envs\tensorflow-gpu\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
import os
key = os.environ.get('EDA_FINAL')


In [3]:
import cohere
from cohere.responses.classify import Example
co = cohere.Client('key')

In [4]:
# read in the data
accidents_data = pd.read_csv('processed_us_car_accidents_2022.csv')

# show columns
print(accidents_data.columns)

Index(['Unnamed: 0', 'ID', 'Source', 'Severity', 'Start_Time', 'End_Time',
       'Start_Lattitude', 'Start_Longitude', 'End_Lattitude', 'End_Longitude',
       'Distance(mi)', 'Description', 'Street', 'City', 'County', 'State',
       'Zipcode', 'Country', 'Timezone', 'Weather_Timestamp', 'Temperature(F)',
       'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)',
       'Wind_Direction', 'Wind_Speed(mph)', 'Precipitation(in)',
       'Weather_Condition', 'Traffic_Signal', 'Sunrise_Sunset', 'State_Abbrv',
       'Population'],
      dtype='object')


Used the GPT4 to generate a list of 50 exmaple whether the accident happened on the highway or not, if yes it will label the direction of the accident as North, South, East, West, if not it will label as Not_Highway.
Then, the exmamples will be used to train a classifier to predict whether the accident happened on the highway or not.

In [5]:
# Randomly selecting 50 descriptions from the dataset
random_descriptions = accidents_data['Description'].sample(50, random_state=1)

random_descriptions = random_descriptions.tolist()

Now these description is passed to the GPT4 model to generate example, and here is the prompt I used:

```
Please use the description column, find out whether the accident is happen on a high way, if yes return the orientation(N,S,W,E) of that accident happened, and if not on a highway or the direction it is not stated in the discription return not_highway. Read a random 50 description, return a list of data with the following structure

examples=[
  Example("description", "North"),
  Example("'description'", "South"),
  Example("description", "East"),
  Example("'description'", "West"),
  Example("description", "North"),
  Example("'description'", "Not_Highway"),
]

When you analyze the description, do not use code, use your own judgment to make the prediction. 


```

In [6]:
# here are the examples generated from the GPT4
examples = [
    Example("Incident on AZ-79 SB near FANNIN-MCFARLAND AQUEDUCT", "South"),
    Example("Left lane blocked due to crash on I-495 Southbound", "South"),
    Example("Lane blocked due to crash on FL-574 Dr Martin Luther King Jr Blvd Eastbound at I-4", "East"),
    Example("Slow traffic on I-405 N - San Diego Fwy N from Venice Blvd/Exit 50 to I-10/Exit 53B", "North"),
    Example("Incident on TOPANGA CANYON BLVD NB near RODEO GROUNDS DR", "North"),
    Example("Two lanes blocked due to crash on I-678 Northbound at Exit 1B I-495 Long Island Expy", "North"),
    Example("Incident on US-378 near HOLLEY FERRY RD", "Not_Highway"),
    Example("Stationary traffic on US-54 S - US-54 N from 375 Loop to Montana Ave/Exit 23", "Not_Highway"),
    Example("Incident on GEORGE WASHINGTON MEMORIAL PKY near VA-123 Chain Bridge Rd", "Not_Highway"),
    Example("Center lane blocked due to crash on I-84 Eastbound at Exit 57 I-91", "East"),
    Example("Stationary traffic on US-12 E - I-94 E - I-35E N from MN-55 to I-494", "East"),
    Example("Slow traffic on I-95 S from Hypoluxo Rd/Exit 60 to Lantana Rd/Exit 61", "South"),
    Example("Lane blocked due to crash on I-85 Northbound at Exit 120 Hamilton Mill Rd", "North"),
    Example("Incident on I-87 NB near I-87", "North"),
    Example("Incident on MN-77 near DIAGONAL BLVD", "Not_Highway"),
    Example("Stationary traffic from Poole Rd (Hodge Rd) to Exit 14", "Not_Highway"),
    Example("Incident on MORNING GLORY AVE near CHERRYDALE DR", "Not_Highway"),
    Example("Crash on the left shoulder on I-17 Northbound at Exit 214B Jomax Rd", "North"),
    Example("Accident on S Holt Rd from Whiskey Slough Rd to I-5", "Not_Highway"),
    Example("Incident on I-575 SB near BARRETT PKY", "South"),
    Example("Stationary traffic on VA-3 - VA-14 from Burkes Pond Rd to John Clayton Memorial Hwy", "Not_Highway"),
    Example("Incident on I-55 SB near US-278 Road closed. Take alternate route", "South"),
    Example("Incident on W GARNETTE ST near W GARNETTE ST E", "Not_Highway"),
    Example("Incident on HICKS RD near ALAMITOS RD", "Not_Highway"),
    Example("Right lane blocked due to accident on I-285 Northbound at Exit 25 US-19 Roswell Rd", "North"),
    Example("Slow traffic on I-10 E from US-98/US-90/Old Spanish Trail to FL-292/Sorrento Rd/Exit 7", "East"),
    Example("Stationary traffic from S Vermont Ave (W Gage Ave) to I-110/Harbor Fwy/Exit 18", "Not_Highway"),
    Example("Incident on CA-132 near N DAKOTA AVE Right shoulder blocked", "Not_Highway"),
    Example("Incident on CA-156 near MONTEREY ST Road closed", "Not_Highway"),
    Example("Incident on TX-21 WB near FM-1209 Road closed. Take alternate route", "West"),
    Example("Incident on N 7TH ST near E DEER VALLEY RD Expect delays", "Not_Highway"),
    Example("Left lane blocked and left hand shoulder blocked due to crash on I-75 Northbound at Exit 250 10th St/14th St", "North"),
    Example("Incident on VESTRY DR near LAMPWICK LN Expect delays", "Not_Highway"),
    Example("Stationary traffic from Blaine St exit [33] to NJ-17", "Not_Highway"),
    Example("Stationary traffic on US-29 N - US-29 S from Timberwood Blvd to Hydraulic Rd", "Not_Highway"),
    Example("Slow traffic on San Bernardino Fwy E - I-10 E from Sierra Ave/Exit 68 to Cherry Ave/Exit 71", "East"),
    Example("Crash on NY 7 eastbound at Exit to I-787; Exit 9E", "East"),
    Example("Lane blocked due to crash on I-65 Southbound at Exit 13A I-40", "South"),
    Example("Incident on WI-57 SB near COUNTY ROAD PP Road closed", "South"),
    Example("Slow traffic on I-696 E from Hoover Rd/Exit 24 to Mound Rd/Exit 23", "East"),
    Example("Incident on I-90 near I-90 Left lane blocked", "Not_Highway"),
    Example("Slow traffic on Garden State Pkwy S from US-9 Exit 123 to Cheesequake Rd Exit 120", "South"),
    Example("Incident on TEXAS ST near MONROE AVE", "Not_Highway"),
    Example("Incident on N LABURNUM AVE near WATTS LN", "Not_Highway"),
    Example("Slow traffic on I-90 W from 22nd St/Exit 172 (I-94 W/I-90 W) to Cermak Rd", "West"),
    Example("Slow traffic on San Diego Fwy S - I-405 S from Getty Center Dr to I-10/Santa Monica Fwy", "South"),
    Example("Left lane blocked due to crash on MA-24 Southbound at Exit 18B I-495 North", "South"),
    Example("Incident on I-80 EB near I-80", "East"),
    Example("Slow traffic on US-301 N - FL-70 from 63rd Ave E to 33rd St E", "North"),
    Example("Incident on SW 59TH LN near HOUSE 13420", "Not_Highway")
]


In [7]:
# apply co.classify to the actual data and append the results to the dataframe
accidents_data['direction'][:10] = accidents_data['Description'][:10].apply(lambda x: co.classify(x, examples))

CohereAPIError: invalid api token